In [1]:
import torch.nn as nn 
import numpy as np 
import torch

import torch.nn.functional as F 
import math, copy , time 
from torch.autograd import Variable
import matplotlib.pyplot as plt 

In [2]:
from MyTransformers import Batch 
def data_gen(V,batch,nbatches):
    'Generate random data for a src-tgt copy task.'
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1 # set <s> as first element
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        yield Batch(src, tgt,0)

In [13]:
from MyTransformers import LabelSmoothing,make_model,NoamOpt,run_epoch,SimpleLossCompute
V = 11 
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.01)
model = make_model(V,V,N=2,d_model=512, d_ff=2048, h=8, dropout=0.3)
model_opt = NoamOpt(model_size=model.src_embed[0].d_model,
                    factor= 1, 
                    warmup=400,
                    optimizer=torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
for epoch in range(10):
    model.train()
    run_epoch(data_iter=data_gen(V,30,20),
              model=model,
              loss_compute=SimpleLossCompute(generator=model.generator,
                                              criterion=criterion,opt=model_opt))


    model.eval()
    print(run_epoch(data_iter=data_gen(V,30,5),
                    model=model,
                    loss_compute=SimpleLossCompute(generator=model.generator,
                                                    criterion=criterion,opt=None)))

Epoch Step: 1 Loss: -10.468163 Tokens per Sec: 1364.029175
Epoch Step: 1 Loss: -12.888931 Tokens per Sec: 2313.796143
tensor(-12.8900)
Epoch Step: 1 Loss: -12.573231 Tokens per Sec: 1729.248047
Epoch Step: 1 Loss: -13.301868 Tokens per Sec: 2468.878174
tensor(-13.3061)
Epoch Step: 1 Loss: -13.043589 Tokens per Sec: 1804.478882
Epoch Step: 1 Loss: -13.423935 Tokens per Sec: 2430.708252
tensor(-13.4216)
Epoch Step: 1 Loss: -13.190002 Tokens per Sec: 1819.400269
Epoch Step: 1 Loss: -13.440957 Tokens per Sec: 2424.165039
tensor(-13.4408)
Epoch Step: 1 Loss: -13.220473 Tokens per Sec: 1831.086670
Epoch Step: 1 Loss: -13.483849 Tokens per Sec: 2319.231201
tensor(-13.4840)
Epoch Step: 1 Loss: -13.250735 Tokens per Sec: 1842.858032
Epoch Step: 1 Loss: -13.481214 Tokens per Sec: 2515.634766
tensor(-13.4810)
Epoch Step: 1 Loss: -13.277815 Tokens per Sec: 1797.797119
Epoch Step: 1 Loss: -13.487893 Tokens per Sec: 2489.611572
tensor(-13.4879)
Epoch Step: 1 Loss: -13.260745 Tokens per Sec: 1803.862

In [15]:
from MyTransformers import subsequent_mask
def greedy_decode(model, src, src_mask,max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, Variable(ys), Variable(subsequent_mask(ys.size(1)).type_as(src.data)))
        prob = model.generator(out[:, -1])
        print(i,prob)
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        print(ys)
    return ys
model.eval()
src = Variable(torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]))
src_mask = Variable(torch.ones(1, 1, 10))
print(greedy_decode(model, src, src_mask, max_len=10, start_symbol=1))

0 tensor([[-11.8144,  -2.2506,  -2.1323,  -2.3044,  -2.3096,  -2.4100,  -2.4081,
          -2.1976,  -2.2582,  -2.2997,  -2.5098]],
       grad_fn=<LogSoftmaxBackward0>)
tensor([[1, 2]])
1 tensor([[-11.9322,  -2.2235,  -2.1459,  -2.3279,  -2.3237,  -2.3995,  -2.4132,
          -2.1613,  -2.2526,  -2.3223,  -2.5163]],
       grad_fn=<LogSoftmaxBackward0>)
tensor([[1, 2, 2]])
2 tensor([[-11.9398,  -2.2198,  -2.1548,  -2.3203,  -2.3282,  -2.4041,  -2.4141,
          -2.1597,  -2.2549,  -2.3128,  -2.5171]],
       grad_fn=<LogSoftmaxBackward0>)
tensor([[1, 2, 2, 2]])
3 tensor([[-11.9444,  -2.2165,  -2.1579,  -2.3222,  -2.3259,  -2.4075,  -2.4215,
          -2.1575,  -2.2515,  -2.3125,  -2.5136]],
       grad_fn=<LogSoftmaxBackward0>)
tensor([[1, 2, 2, 2, 7]])
4 tensor([[-11.7707,  -2.2121,  -2.1534,  -2.3234,  -2.3043,  -2.4408,  -2.3837,
          -2.1928,  -2.2736,  -2.2749,  -2.5254]],
       grad_fn=<LogSoftmaxBackward0>)
tensor([[1, 2, 2, 2, 7, 2]])
5 tensor([[-11.9389,  -2.2145,  -2.